In [ ]:
import torch
from torch import nn
import pandas as pd
from torch.utils.data import TensorDataset,Dataset,DataLoader,random_split

def normalize(dataset):
    dataNorm=((dataset-dataset.min())/(dataset.max()-dataset.min()))
    dataNorm["CreditLevel"]=dataset["CreditLevel"]-1
    return dataNorm

# data preprocessing
train_dataset = pd.read_csv("BankChurners.csv")
print(f"Original data frame has {train_dataset.shape[0]} rows and {train_dataset.shape[1]} columns.")
train_dataset = train_dataset.drop(['CustomerId','Geography'],axis=1)
train_dataset = normalize(train_dataset)
train_dataset.info()
print(f"preprocessing data frame has {train_dataset.shape[0]} rows and {train_dataset.shape[1]} columns.")
train_dataset.head()
pd.set_option('precision',3)

In [ ]:
x_train = torch.tensor(train_dataset.drop('CreditLevel',axis=1).values.astype('float32'))
y_train = torch.tensor(train_dataset['CreditLevel'].values)
train_tensor = TensorDataset(x_train,y_train)
n_train = int(len(train_tensor)*0.8)
n_valid = len(train_tensor) - n_train
ds_train,ds_valid = random_split(train_tensor,[n_train,n_valid])
dl_train = DataLoader(ds_train,batch_size = 100,shuffle=True)
dl_valid_batch_size = 30
dl_valid = DataLoader(ds_valid,batch_size = dl_valid_batch_size,shuffle=False)

for features,labels in dl_train:
    print(features,labels)
    break

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.init import kaiming_uniform_,xavier_uniform_

class Net(nn.Module):
    def __init__(self,n_inputs):
        super(Net, self).__init__()
        # input to first hidden layer
        self.hidden1 = nn.Linear(n_inputs, 14)
        kaiming_uniform_(self.hidden1.weight, nonlinearity='relu')
        self.dropout1 = nn.Dropout(0.3)
        self.act1 = nn.ReLU()
        
        # second hidden layer
        # self.hidden2 = nn.Linear(14, 10)
        # kaiming_uniform_(self.hidden2.weight, nonlinearity='relu')
        # self.dropout2 = nn.Dropout(0.3)
        # self.act2 = nn.ReLU()
        
        # third hidden layer and output
        self.hidden3 = nn.Linear(14, 10)
        xavier_uniform_(self.hidden3.weight)
        #self.dropout3 = nn.Dropout(0.3)
        self.act3 = nn.Softmax(dim=1)
    
    def forward(self,X):
         # input to first hidden layer
        X = self.hidden1(X)
        X = self.dropout1(X)
        X = self.act1(X)
        
        # second hidden layer
        #X = self.hidden2(X)
        #X = self.dropout2(X)
        #X = self.act2(X)
        
        # output layer
        X = self.hidden3(X)
        #X = self.dropout3(X)
        X = self.act3(X)
        return X

model = Net(7)
print(model)

In [ ]:
#define a Loss function and optimizer
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
# Train the network
num_epochs = 20
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (inputs,targets) in enumerate(dl_train):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs,targets)
        loss.backward()
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
        if i % 100 == 0:   
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

In [ ]:
# model test
model.eval()
accuracy = 0.0
total = 0.0
with torch.no_grad():
    for data in dl_valid:
        inputs,targets = data
        outputs = model(inputs)
        _,predicted = torch.max(outputs.data,1)
        total += targets.size(0)
        accuracy += (predicted == targets).sum().item()

accuracy = (100*accuracy/total)
print(accuracy)

In [ ]:
def testClassess():
    number_of_labels = 10
    class_correct = list(0. for i in range(number_of_labels))
    class_total = list(0. for i in range(number_of_labels))
    with torch.no_grad():
        for data in dl_valid:
            inputs,targets = data
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            c = (predicted == targets).squeeze()
            for i in range(dl_valid_batch_size):
                target = targets[i]
                class_correct[target] += c[i].item()
                class_total[target] += 1

    for i in range(number_of_labels):
        print('Accuracy of %5d : %2d %%' % (
            i, 100 * class_correct[i] / class_total[i]))
        
        
testClassess()